In [1]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [66]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import pickle

In [67]:
with open(r"D:\Resources\GenAI\GPT from scratch\data.txt",'r',encoding='utf-8') as f:
  text_data = f.read()

In [68]:
text_data

"my name is sudhanshu kumar , i work with euron , Helping Millions of Students Succeed\nSudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.\n\nIn 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordable and accessible tech courses in the world.\nHelping Millions of Students Succeed\nSudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has h

In [69]:
# creating embeddings
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000,oov_token='<OOV>')
tokenizer.fit_on_texts([text_data])

In [70]:
tokenizer

In [71]:
sequence = tokenizer.texts_to_sequences([text_data])[0]

In [72]:
len(sequence)

860

In [73]:
with open("tokenizer.pkl",'wb') as f:
  pickle.dump(tokenizer,f)

In [74]:
max_seq_length = 100
def create_dataset(seq,window_size = max_seq_length):
    input ,labels = [],[]
    for i in range(len(seq) -window_size):
        input.append(seq[i:i+window_size])
        labels.append(seq[i+1:i+window_size+1])
    return np.array(input) , np.array(labels)

In [75]:
x_data, y_data = create_dataset(sequence)

In [76]:
len(x_data)

760

In [77]:
# input
x_data[0]

array([158, 159,  16,   9, 160, 161, 162,  20,  21,  69,  49,   5,  22,
        28,  36,  70,   2,  23,  10,  71,  29,   4,  72,  73,  12,   8,
        74,  24,  30,   3,  50,  11,  75,  76,  30,  77,  78,  51,  22,
        14,  79,  80,  81,  82,  20,   3,  31,  37,  83,   2,  28,   6,
        84,  85,  86,  87,  38,   5,  88,  22,  39,   9,  52,  53,  14,
        54,  40,  37,  89,  11,  90,   4,  91,  25,   2,  92,  93,  18,
        94,   6,  95,  11,  12,  96,  26,  97,   6,   4,  98,  99, 100,
       101,  55,  27,  41,  12,   4, 102, 103,   9])

In [78]:
# This is the output which the input tries to learn
y_data[0]

array([159,  16,   9, 160, 161, 162,  20,  21,  69,  49,   5,  22,  28,
        36,  70,   2,  23,  10,  71,  29,   4,  72,  73,  12,   8,  74,
        24,  30,   3,  50,  11,  75,  76,  30,  77,  78,  51,  22,  14,
        79,  80,  81,  82,  20,   3,  31,  37,  83,   2,  28,   6,  84,
        85,  86,  87,  38,   5,  88,  22,  39,   9,  52,  53,  14,  54,
        40,  37,  89,  11,  90,   4,  91,  25,   2,  92,  93,  18,  94,
         6,  95,  11,  12,  96,  26,  97,   6,   4,  98,  99, 100, 101,
        55,  27,  41,  12,   4, 102, 103,   9, 104])

In [79]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_len, d_model, **kwargs):  # <-- add **kwargs
        super().__init__(**kwargs)                   # <-- pass kwargs here too
        self.max_len = max_len
        self.d_model = d_model

        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(d_model)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        self.pos_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

    def get_config(self):
        config = super().get_config()
        config.update({
            "max_len": self.max_len,
            "d_model": self.d_model
        })
        return config


In [80]:
def transformer_block(embed_dim, num_heads, ff_dim, dropout = 0.1):

  # input layer
  inputs = layers.Input(shape = (None,embed_dim))
  # Multi head attention
  attn_output = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)(inputs,inputs)
  # Dropout
  attn_output = layers.Dropout(dropout)(attn_output)
  # Layer Normalization
  out1 = layers.LayerNormalization(epsilon=1e-6)(inputs + attn_output)

  # Feed Forward Neural network
  ffn = tf.keras.Sequential([
   layers.Dense(ff_dim,activation='relu'),
   layers.Dense(embed_dim)
  ])

  ffn_output = ffn(out1)
  ffn_output = layers.Dropout(dropout)(ffn_output)
  out2 = layers.LayerNormalization(epsilon=1e-6)(out1+ffn_output)
  return tf.keras.Model(inputs = inputs, outputs = out2)


In [81]:
vocab_size = 5000
max_seq_len = 100
embed_dim = 128
num_heads = 4
ff_dim = 512
num_layers = 4
batch_size = 8
epoch  = 25

def build_gpt_model():

  # input
  inputs = layers.Input(shape=(max_seq_len,))
  x = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim)(inputs)
  x = PositionalEncoding(max_seq_len,embed_dim)(x)

  # stack of transformer
  for _ in range(num_layers):
    x = transformer_block(embed_dim=embed_dim,num_heads=num_heads, ff_dim=ff_dim)(x)

  outputs = layers.Dense(vocab_size, activation='softmax')(x)

  return tf.keras.Model(inputs,outputs)


In [82]:
model = build_gpt_model()
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [83]:
model.fit(x_data,y_data,batch_size=batch_size,epochs=epoch,validation_split=0.10)

Epoch 1/25
86/86 [==============================] - 10s 41ms/step - loss: 5.7591 - accuracy: 0.0304 - val_loss: 7.3054 - val_accuracy: 0.0468
Epoch 2/25
86/86 [==============================] - 3s 33ms/step - loss: 5.3388 - accuracy: 0.0325 - val_loss: 7.3037 - val_accuracy: 0.0468
Epoch 3/25
86/86 [==============================] - 3s 33ms/step - loss: 5.3381 - accuracy: 0.0326 - val_loss: 7.5771 - val_accuracy: 0.0468
Epoch 4/25
86/86 [==============================] - 3s 33ms/step - loss: 5.3374 - accuracy: 0.0328 - val_loss: 7.4269 - val_accuracy: 0.0437
Epoch 5/25
86/86 [==============================] - 3s 33ms/step - loss: 5.3811 - accuracy: 0.0329 - val_loss: 7.5479 - val_accuracy: 0.0468
Epoch 6/25
86/86 [==============================] - 3s 32ms/step - loss: 5.3333 - accuracy: 0.0327 - val_loss: 7.3975 - val_accuracy: 0.0468
Epoch 7/25
86/86 [==============================] - 3s 32ms/step - loss: 5.3329 - accuracy: 0.0331 - val_loss: 7.5705 - val_accuracy: 0.0468
Epoch 8/25
8

In [84]:
model.save('gpt_test_genai_class.h5')

In [85]:
model.summary()

Model: "model_121"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_118 (InputLayer)      [(None, 100)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 100, 128)          640000    
                                                                 
 positional_encoding_5 (Posi  (None, 100, 128)         0         
 tionalEncoding)                                                 
                                                                 
 model_117 (Functional)      (None, None, 128)         396032    
                                                                 
 model_118 (Functional)      (None, None, 128)         396032    
                                                                 
 model_119 (Functional)      (None, None, 128)         396032    
                                                         

In [86]:
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load trained model
model = tf.keras.models.load_model("gpt_test_genai_class.h5", custom_objects={"PositionalEncoding": PositionalEncoding})

# Load tokenizer
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Parameters
max_seq_len = 100

In [87]:
def generate_text(seed_text, model, tokenizer, num_tokens=50, temperature=1.0):
    for _ in range(num_tokens):
        token_seq = tokenizer.texts_to_sequences([seed_text])[0]
        token_seq = token_seq[-max_seq_len:]  # Trim to max length
        padded_seq = pad_sequences([token_seq], maxlen=max_seq_len)

        preds = model.predict(padded_seq, verbose=0)[0, -1]  # Get prediction for last time step
        preds = np.asarray(preds).astype('float64')

        # Apply temperature sampling
        preds = np.log(preds + 1e-9) / temperature
        preds = np.exp(preds) / np.sum(np.exp(preds))

        next_token_id = np.random.choice(len(preds), p=preds)
        next_word = tokenizer.index_word.get(next_token_id, '')

        seed_text += ' ' + next_word

        if next_word == '':  # Optional: break if OOV or unknown word
            break

    return seed_text

In [88]:
seed_text = "who is sudhanshu"
generated = generate_text(seed_text, model, tokenizer, num_tokens=50, temperature=1.0)

print("📝 Generated Text:")
print(generated)

📝 Generated Text:
who is sudhanshu in to most their from to and at they is while the the is gain designed to support 5 a during a thanks sudhanshu significant ineuron a the wipro he sudhanshu's and power importantly books reach the truth ineuron bundle in time due engineering learn the offer a very like
